<a href="https://colab.research.google.com/github/devikaranam/FMML-Lab-Assignment/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# **1st answer**

The size of the validation set in machine learning can have a significant impact on the accuracy and generalization of a model. Let's explore how increasing or reducing the percentage of the validation set can affect accuracy:

1. **Increasing the Percentage of the Validation Set:**
   - **Pros:** When you increase the size of the validation set, you typically get a more reliable estimate of how well your model will generalize to unseen data. A larger validation set provides a better representation of the data distribution, which can help in detecting overfitting.
   - **Cons:** With a larger validation set, you have fewer data points available for training your model. This might lead to a less complex model and potentially lower training accuracy. It can also make training take longer due to the reduced training data.

   In summary, increasing the percentage of the validation set can improve your model's ability to generalize but may result in slightly lower training accuracy.

2. **Reducing the Percentage of the Validation Set:**
   - **Pros:** A smaller validation set allows you to use more data for training, which can lead to a more complex model and potentially higher training accuracy. Training may also be faster with a smaller validation set.
   - **Cons:** With a smaller validation set, you have a less reliable estimate of your model's generalization performance. The model might overfit to the training data since there is less data available for validation. This can lead to poor performance on unseen data.

   In summary, reducing the percentage of the validation set can lead to higher training accuracy but at the cost of potentially worse generalization to unseen data.

It's essential to strike a balance between the size of the training and validation sets. The choice of validation set size depends on the specific problem, the amount of available data, and the trade-off between training and validation performance. Techniques like cross-validation can also be used to mitigate the impact of small validation sets.

In practice, it's common to split the data into training, validation, and test sets, with a typical split like 70% for training, 15% for validation, and 15% for testing. This allocation provides a good compromise between training performance and the ability to estimate generalization performance accurately. However, these percentages can vary based on the dataset's characteristics and the problem you're trying to solve.

# **2nd answer**

The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. The key factor here is the representativeness of the validation set and its ability to provide a reliable estimate of the model's performance on unseen data (test set). Here's how different scenarios can impact this prediction:

1. **Large Validation Set, Small Training Set:**
   - If you have a large validation set and a small training set, the validation set is more likely to represent the overall data distribution better. As a result, the performance on the validation set can be a more accurate predictor of how the model will perform on the test set.
   - However, having a small training set may limit the model's ability to generalize well, which can lead to a lower test set accuracy.

2. **Small Validation Set, Large Training Set:**
   - A small validation set might not be representative enough of the overall data distribution. This can lead to higher variability in the validation set's performance, making it less reliable as a predictor of test set accuracy.
   - A large training set provides more data for the model to learn from, potentially resulting in better generalization and higher test set accuracy.

3. **Balanced Split (e.g., 70% training, 15% validation, 15% test):**
   - This balanced split often strikes a reasonable compromise. The validation set, although smaller than the training set, is still sufficiently representative to provide a decent estimate of the model's performance on unseen data.
   - The training set, being the largest portion, allows the model to learn well.

4. **Cross-Validation:**
   - Cross-validation techniques, such as k-fold cross-validation, can help mitigate the impact of small validation sets. By repeatedly splitting the data into different training and validation sets and averaging the results, you get a more robust estimate of model performance.

In summary, the size of the training and validation sets can influence how accurately the validation set predicts the model's performance on the test set. A larger, more representative validation set tends to provide a more reliable estimate, but it should be balanced with having enough data in the training set for the model to learn effectively. It's essential to choose a split that aligns with the specific characteristics of your dataset and the goals of your machine learning project. Cross-validation can also be a valuable technique for obtaining a more robust estimate of model performance.

# **3rd answer**

The choice of the percentage to reserve for the validation set depends on various factors, including the size of your dataset, the complexity of your model, and your specific goals. However, there are some common practices that can help strike a balance between having a representative validation set and a sufficiently large training set. A 70-30 or 80-20 split is often a good starting point, where:

1. **Training Set:** 70% or 80% of the data is allocated to the training set. This provides a substantial amount of data for your model to learn from, which is essential, especially for complex models or when you have a limited dataset.

2. **Validation Set:** 30% or 20% of the data is reserved for the validation set. This size is typically large enough to provide a representative sample of the data distribution, allowing you to estimate how well your model generalizes to unseen data.

These splits are common because they provide a reasonable compromise between having a sufficiently large training set and a representative validation set. However, the exact split can vary depending on the dataset's size and characteristics.

In cases where you have a very large dataset, you might be able to allocate a smaller percentage to the validation set, as even a smaller fraction can be representative. Conversely, if you have a very limited dataset, you may need to allocate a larger percentage to the validation set to ensure a reliable estimate of model performance.

It's essential to experiment with different splits and use techniques like cross-validation to assess how your model's performance varies with different validation set sizes. The goal is to find a split that allows your model to learn effectively while still providing a meaningful estimate of its generalization performance on the test set.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# **1st Answer**

Averaging validation accuracy across multiple splits can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly known as cross-validation, and it helps address issues related to the randomness of the data split into training and validation sets. Here's why it's beneficial:

Reduced Variance: When you split your dataset into a single training set and a single validation set, the performance of your model can be sensitive to the specific data points in those sets. Averaging results over multiple splits reduces this sensitivity, making the evaluation more stable.

Better Generalization Estimate: By repeatedly training and evaluating the model on different subsets of the data, cross-validation provides a more robust estimate of how well your model is likely to generalize to unseen data. It reduces the risk of overfitting to a particular validation set.

Improved Confidence Intervals: When you calculate the mean and standard deviation of validation accuracy over multiple splits, you can also compute confidence intervals. This helps you understand the range of possible model performances and provides a measure of your confidence in the reported accuracy.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation, where the dataset is divided into 'k' subsets or folds, and the model is trained and validated 'k' times, with each fold used as the validation set once. The final evaluation metric is typically the average of the individual validation results.
n summary, averaging validation accuracy across multiple splits through cross-validation is a standard practice in machine learning because it provides more consistent and reliable results, helping you make better decisions about your model's performance and generalization ability.



# **2nd answer**


Cross-validation, which involves averaging validation accuracy across multiple splits, does not directly provide a more accurate estimate of test accuracy. Instead, it provides a more accurate estimate of how well your model is likely to generalize to unseen data, which is what test accuracy aims to measure.

Here's the distinction:

Validation Accuracy: During cross-validation, you partition your dataset into training and validation sets multiple times. You train your model on the training set and evaluate it on the validation set in each fold. The purpose of this is to assess how well your model is performing on data it hasn't seen during training. Averaging these validation accuracies provides a more reliable estimate of how your model might perform on unseen data from the same distribution as your training data.

Test Accuracy: Test accuracy is the measure of how well your model performs on a completely independent dataset that it has never encountered during training or cross-validation. This dataset is used to assess how well your model generalizes to entirely new and unseen examples. Test accuracy is a critical metric because it tells you how well your model is likely to perform in real-world scenarios.

While cross-validation gives you a better understanding of your model's performance on your available data and helps you tune hyperparameters and make model selection decisions, it doesn't replace the need for a separate, held-out test set. The test set provides an unbiased estimate of how your model will perform in production or on new, unseen data.
So, cross-validation doesn't directly estimate test accuracy, but it indirectly helps you build models that are more likely to perform well on unseen data by giving you a better sense of your model's generalization capabilities. To get an accurate estimate of test accuracy, you should reserve a portion of your data specifically for testing and evaluate your model on it only once, after all model development and hyperparameter tuning are complete.



# **3rd answer**

The number of iterations (or folds) in cross-validation can have an impact on the estimate of model performance. However, there is a trade-off involved, and simply increasing the number of iterations does not necessarily guarantee a better estimate. Here are some key points to consider:

Bias-Variance Trade-Off: Cross-validation involves partitioning your data into training and validation sets multiple times (k times in k-fold cross-validation). The choice of 'k' affects the bias-variance trade-off:

Smaller 'k' (e.g., 5 or 10) results in a higher variance estimate of model performance. This is because each validation set is smaller, and the model's performance can vary significantly depending on the specific validation data.
Larger 'k' results in a lower variance estimate but may introduce bias because each validation set is larger, and it may not capture variations in the data as effectively.
Computation Time: Increasing the number of iterations increases the computational cost of cross-validation. This can be a concern, especially with large datasets or complex models. You need to strike a balance between computational resources and the quality of the estimate.

Stability of Estimate: In practice, you often see diminishing returns in terms of estimate stability as you increase the number of iterations. For example, going from 5-fold to 10-fold cross-validation may provide a noticeable improvement in estimate stability, but going from 10-fold to 20-fold may not provide as much additional benefit.

Stratification: In some cases, you may want to use stratified cross-validation, especially if your dataset has class imbalance. Stratified cross-validation ensures that each fold maintains the same class distribution as the whole dataset. This can be important for accurate estimates, and you may need to consider this when choosing the number of iterations.

In summary, there is no one-size-fits-all answer to the question of whether a higher number of iterations leads to a better estimate in cross-validation. The choice of 'k' (the number of iterations) should be made based on factors such as the dataset size, computational resources, and the desired trade-off between bias and variance in the estimate. It's often a good practice to start with a reasonable 'k', evaluate the results, and consider increasing it if necessary to achieve more stable estimates, but be mindful of the potential computational costs

# **4th answer**

4rth answer
Increasing the number of iterations in cross-validation can help mitigate some of the challenges posed by having very small training or validation datasets, but it doesn't necessarily solve all the problems associated with small datasets. Here are some considerations:

Small Training Dataset:
Advantage of Increasing Iterations: When you have a small training dataset, each training fold in cross-validation contains a limited amount of data for model training. Increasing the number of iterations (e.g., using a higher 'k' in k-fold cross-validation) allows the model to be trained on different subsets of the data, which can help it capture more variations in the data and potentially improve the model's generalization.

Limitation: While increasing iterations can help, it doesn't change the fundamental limitation of having a small training dataset. If your training data is very small, the model might still struggle to learn complex patterns, and the estimated performance might be overly optimistic.

Small Validation Dataset:
Advantage of Increasing Iterations: With a small validation dataset, the estimated model performance can be noisy and less reliable. Increasing the number of iterations can provide a more stable estimate of model performance because it averages results over multiple validation sets.

Limitation: Similar to addressing a small training dataset, increasing iterations doesn't fundamentally change the fact that you have limited validation data. The estimated performance may still be sensitive to the specific data points in the validation sets.

In cases of extremely small datasets, it's important to be cautious about overfitting, as models can easily memorize the training data when it's small. You might consider using simpler models, regularization techniques, or data augmentation (if applicable) to help improve generalization.

In summary, increasing the number of iterations in cross-validation is a good practice when dealing with small datasets as it can provide more stable estimates of model performance. However, it doesn't eliminate the inherent challenges of having small amounts of training or validation data. Careful model selection, regularization, and data augmentation strategies are also essential considerations when working with very limited data.